In [1]:
import torch.nn as nn
import torch_geometric.nn as nng
import torch
import random
#from torch_geometric.utils import add_self_loops #ADDED!
import torch.nn.functional as F #ADDED

from typing import List, Optional, Tuple, Union
import torch.nn.functional as F
from torch import Tensor
# from torch.nn import LSTM
# from torch_sparse import SparseTensor, matmul
# from torch_geometric.nn.aggr import Aggregation, MultiAggregation
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.dense.linear import Linear
from torch_geometric.typing import Adj, OptPairTensor, Size
import torchvision.ops

from torch_geometric.data import Data, DataLoader, DataListLoader
from torch.utils.tensorboard import SummaryWriter
import time
import numpy as np

In [9]:
# Build the network as described in (https://github.com/cfl-minds/gnn_laminar_flow)
# but using pyTorch

###############################
### EdgeSmoothing CLASS DEF ###
###############################
#@title EdgeSmoothing layer codes
class EdgeSmoothing(nng.conv.MessagePassing): #EdgeSmoothing(nng.conv.GCNConv):
    '''Convolutions class as described in paper, built on GCNConv, 
    with SGCN and FVnew adaptations'''
    def __init__(self,**kwargs):
        kwargs.setdefault('aggr', 'mean')
        super().__init__(**kwargs)

    #def forward(self, x: Tensor, edge_index: Adj, edge_weight: OptTensor = None) -> Tensor:
    def forward(self, x, edge_index):

        # propagate_type: (x: Tensor, edge_weight: OptTensor)
        #out = self.propagate(edge_index, x=x, edge_weight=edge_weight, size=None)
        out = self.propagate(edge_index=edge_index, x=x) #for FVnew

        return out

    #def message(self, x_j: Tensor, edge_weight: OptTensor) -> Tensor:
    #   return x_j if edge_weight is None else edge_weight.view(-1, 1) * x_j
    def message(self, x_j, x_i, edge_feat=None, edge_attr=None):
        """
        x_j [num_edges, label_dim]
        x_i [num_edges, label_dim]
        """
        symmetric = (x_j + x_i)/2 #paper: symmetric node features
        return symmetric

    #def message_and_aggregate(self, adj_t: SparseTensor, x: Tensor) -> Tensor:
    #    return spmm(adj_t, x, reduce=self.aggr)

    def update(self, aggr_out, x_j):
        """
        aggr_out [num_nodes, label_dim, out_channels]
        """
        return aggr_out

**Models 1 and 2**
----

In [3]:
# Build the network as described in (https://github.com/cfl-minds/gnn_laminar_flow)
# but using pyTorch

class swish(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
    def forward(self, x):
        return x * torch.sigmoid(x)

#################################
## InvariantEdgeConv CLASS DEF ##
#################################
#@title InvariantEdgeConv codes, with node_attr_num and edge_attr_num
class InvariantEdgeConv(nng.conv.MessagePassing): #InvariantEdgeConv(nng.conv.GCNConv):
    '''Convolutions class as described in paper, built on GCNConv, 
    with SGCN and FVnew adaptations'''
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        #aggr: Optional[Union[str, List[str], Aggregation]] = 'add',

        old_mlp_ief: int = 0, #paper: previous edge feature dimension
        mlp_hs: int = 128, #paper: two MLPs with hidden size 128 used
        mlp_ief: int = 1, #paper: one MLP with intermediate edge feature dimension fe
        
        node_attr_num: int = 0, #FVnew: number of persistent node_attr
        edge_attr_num: int = 0, #FVnew: number of edge_attr + rel_node_attr
        hidden_size: int = 0, #sGCN: convolution internal hidden size
        dropout: float = 0, #sGCN: dropout
        **kwargs
    ):

        kwargs.setdefault('aggr', 'add')
        super().__init__(**kwargs)

        self.in_channels = in_channels + node_attr_num
        self.out_channels = out_channels
        
        #paper####
        # self.mlp_message = nng.MLP([in_channels*2+old_mlp_ief, mlp_hs, mlp_ief],act='swish', bias=True)
        # self.mlp_update = nng.MLP([mlp_ief+in_channels, mlp_hs, out_channels],act='swish', bias=True)
        self.mlp_message = torchvision.ops.MLP(in_channels*2+old_mlp_ief, [mlp_hs, mlp_ief], \
                                               activation_layer=swish, bias=True)
        self.mlp_update = torchvision.ops.MLP(mlp_ief+in_channels, [mlp_hs, out_channels], \
                                              activation_layer=swish, bias=True)
        # self.mlp_message = torchvision.ops.MLP(in_channels*2+old_mlp_ief, [mlp_hs, mlp_ief], \
        #                                        activation_layer=None, bias=True)
        # self.mlp_update = torchvision.ops.MLP(mlp_ief+in_channels, [mlp_hs, out_channels], \
        #                                       activation_layer=None, bias=True)
        self.new_edge_features = None #torch.tensor([0.])

        #FVnew or sGCN####
        # self.lin_in = torch.nn.Linear(edge_attr_num, hidden_size * in_channels)
        # self.lin_out = torch.nn.Linear(hidden_size * in_channels, out_channels)
        # self.dropout = dropout ####

    #def forward(self, x: Tensor, edge_index: Adj, edge_weight: OptTensor = None) -> Tensor:
    def forward(self, x, edge_index, edge_feat=None, edge_attr=None, node_attr=None):

        if (node_attr is not None): #for FVnew
            x = torch.cat((x,node_attr),dim=1) #persistent node_attr.

        # if edge_feat is None: #for paper: previous edge feat added if existent
        #     edge_feat = torch.zeros((edge_index.size(1),1))
        out = self.propagate(x=x, edge_index=edge_index, edge_feat=edge_feat, edge_attr=edge_attr) #for FVnew

        return out, self.new_edge_features

    #def message(self, x_j: Tensor, edge_weight: OptTensor) -> Tensor:
    #   return x_j if edge_weight is None else edge_weight.view(-1, 1) * x_j
    def message(self, x_j, x_i, edge_feat, edge_attr=None):
        """
        x_j [num_edges, label_dim]
        x_i [num_edges, label_dim]
        edge_attr [num_edges, #attr] or none ##ADDED
        """
        symmetric = (x_j + x_i)/2 #paper: symmetric node features
        asymmetric = abs(x_j - x_i)/2 #instead of raw features x_j and x_i
        if edge_feat is not None:
            new_edge_feat = torch.cat((symmetric,asymmetric,edge_feat),dim=1)
        else:
            new_edge_feat = torch.cat((symmetric,asymmetric),dim=1)
        message = self.mlp_message(new_edge_feat)
        # message = message * F.sigmoid(message) #MLP swish activation if act was None
        self.new_edge_features = message

        ##for FVnew ####
        # if edge_attr is not None: 
        #   scaling = F.relu(self.lin_in(edge_attr))  # [n_edges, hidden_size * in_channels]
        # else:
        #   scaling = torch.ones(x_j.size()).unsqueeze(-1).to(x_j.device)
        # n_edges = x_j.size(0)
        # # [n_edges, in_channels, ...] * [n_edges, in_channels, 1]
        # result = scaling.reshape(n_edges, self.in_channels, -1) * message.unsqueeze(-1)
        ####

        return message #result.view(n_edges, -1)

    #def message_and_aggregate(self, adj_t: SparseTensor, x: Tensor) -> Tensor:
    #    return spmm(adj_t, x, reduce=self.aggr)

    def update(self, aggr_out, x):
        """
        x_j [num_nodes, label_dim, 1]
        aggr_out [num_nodes, label_dim, out_channels]
        """
        aggr_out = torch.cat((x,aggr_out),dim=1) #paper
        aggr_out = self.mlp_update(aggr_out)
        # aggr_out = aggr_out * F.sigmoid(aggr_out) #MLP swish activation if act was None

        ##for sGCN ####
        # aggr_out = self.lin_out(aggr_out)  #[num_nodes, label_dim, out_features]
        # aggr_out = torch.tanh(aggr_out); #aggr_out = F.relu(aggr_out)
        # aggr_out = F.dropout(aggr_out, p=self.dropout, training=self.training)
        ####

        return aggr_out

In [4]:
##################################
## InvariantEdgeModel CLASS DEF ##
##################################
#@title InvariantEdgeModel codes
#https://github.com/cfl-minds/gnn_laminar_flow/blob/main/network_utils.py#L22
class InvariantEdgeModel(nn.Module):
    def __init__(self, input_size=3, input_edge_feat=3, edge_feature_dim=[4,8,16,32,64,64,32,16], \
                 num_filters=[8,16,32,64,64,32,16,8], output_size=3, \
                 node_attr_num=0, edge_attr_num=0, FV_hidden_size=0):
        super().__init__()

        num_filters = [input_size-2] + num_filters
        edge_feature_dim = [input_edge_feat] + edge_feature_dim
        #increase layer input channel size by 2 because of skip links
        self.layers = [InvariantEdgeConv(in_channels = num_filters[i]+2, 
                                         out_channels = num_filters[i+1],
                                         old_mlp_ief = edge_feature_dim[i],
                                         mlp_hs = 128,
                                         mlp_ief = edge_feature_dim[i+1],
                                         node_attr_num = node_attr_num,
                                         edge_attr_num = edge_attr_num,
                                         hidden_size = FV_hidden_size, 
                                         dropout = 0) \
                       for i in range(len(num_filters)-1)]
        self.layers = torch.nn.ModuleList(self.layers)

        self.smooth_layer = EdgeSmoothing()
        self.final_layer = Linear(num_filters[-1]+2, output_size, bias=True)

    def forward(self, data): #just data input for DataParallel from listloader
        x = data.x.clone()
        x_0 = data.x.clone()[:,0:2] #the coordinate data for skip links
        edge_feat = x[data.edge_index,:].mean(0) #size Ex3 or Ex12
        #edge_feat = None #if no initial edge features

        for layer in self.layers:
            x,edge_feat = layer(x, data.edge_index, edge_feat=edge_feat, edge_attr=None, node_attr=None)
            x = self.smooth_layer(x, data.edge_index)
            x = torch.cat((x_0,x),dim=1) #skip links

        x = self.final_layer(x)

        return x, data.y

In [5]:
# define training function
# https://github.com/cfl-minds/gnn_laminar_flow/blob/main/training.py
# https://github.com/cfl-minds/gnn_laminar_flow/blob/main/training_utils.py

##############################
### IVEModelfit CLASS DEF ####
##############################
#@title IVEModelfit CLASS: fit() func
class IVEModelfit:
    def __init__(self, input_size=3, input_edge_feat=0, edge_feature_dim=[4,8,16,32,64,64,32,16], \
                 num_filters=[8,16,32,64,64,32,16,8], output_size=3, \
                 node_attr_num=0, edge_attr_num=0, FV_hidden_size=0, dropout=0, device='cuda:0'):
        self.model = InvariantEdgeModel(input_size = input_size,
                                        input_edge_feat = input_edge_feat,
                                        edge_feature_dim=edge_feature_dim,
                                        num_filters=num_filters,
                                        output_size = output_size,
                                        node_attr_num=0, edge_attr_num=0, FV_hidden_size=0)
        # if torch.cuda.device_count() > 1:
        #     self.model = nn.DataParallel(self.model)
        self.model.to(device); self.device=device; self.mean,self.var = None,None
        self.tb = SummaryWriter() #tensorboard --logdir=runs

    def fit(self, path, dataset, valset, max_epoch=1000, bs=64, lr=0.002, weight_decay=0.002, verbose=True):
        criterion = nn.L1Loss().to(self.device);
        optimizer = torch.optim.Adam(self.model.parameters(), lr=lr, weight_decay=0)
        lambda1 = lambda epoch: 1/(1+epoch*weight_decay) #decay rate is weight decay
        scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lambda1) #lr is lr0*lambda1
        
        self.model.train() #force dropouts to occur.
        start=time.time(); #print(self.model);
        self.tb = SummaryWriter() #TensorBoard to show progress
        scaler = torch.cuda.amp.GradScaler() #ADDED: half pres

        #for epoch in range(max_epoch):
        epoch = 0; early_stop = 0; min_val_loss = torch.inf
        while epoch < max_epoch:
            running_loss=0; num_batches=0
            # create minibatches
            trainLoader = DataLoader(dataset, batch_size=bs, shuffle=True)
            #trainLoader = DataListLoader(dataset, batch_size=bs, shuffle=True)

            for batch_list in trainLoader:
              #batch = batch.to(self.device)
              optimizer.zero_grad() #ADDED

              # do one step of stochastic gradient descent: U=U-lr(dL/dU)
              with torch.cuda.amp.autocast(): #ADDED: half pres
                  batch_list = batch_list.to(self.device)
                  outputs, GTy = self.model(batch_list)
                  loss = criterion(outputs, GTy)

              scaler.scale(loss).backward() #ADDED: half pres
              nn.utils.clip_grad_norm_(self.model.parameters(), 5) #if nan loss!
              scaler.step(optimizer) #ADDED: half pres
              scaler.update() #ADDED: half pres

              # add the loss of this batch to the running loss
              running_loss += loss.detach().item(); num_batches+=1

            # compute stats for the full training set
            scheduler.step() #anneal lr
            total_loss = running_loss/num_batches; elapsed = time.time()-start
            self.tb.add_scalar("Loss/Epoch", total_loss, epoch) #TB plot loss

            if verbose & (epoch%5 == 0):
                #print('Epoch {} loss: {}'.format(epoch+1, loss.item()))
                print('epoch=',epoch,'\t time=',elapsed,'\t loss=',total_loss)
            

            # check stats for validation set, for early stopping
            running_loss=0; num_batches=0; 
            valLoader = DataLoader(valset, batch_size=bs, shuffle=True)
            #valLoader = DataListLoader(valset, batch_size=bs, shuffle=True)
            for batch_list in valLoader:
                with torch.cuda.amp.autocast(): #ADDED: half pres
                    batch_list = batch_list.to(self.device)
                    outputs, GTy = self.model(batch_list)
                    loss = criterion(outputs, GTy)
                running_loss += loss.detach().item(); num_batches+=1
            val_loss = running_loss/num_batches
            self.tb.add_scalar("ValLoss/Epoch", val_loss, epoch) #TB plot loss
            # if not improved in 60 epochs, end training.
            if val_loss < min_val_loss:
                early_stop = 0; min_val_loss = val_loss
                with open(path+'IVEModel_checkpoint_'+str(epoch)+'.txt', 'wb') as f:
                    torch.save(self.model, f) #save model checkpoint
            else:
                early_stop += 1
            epoch = max_epoch+1 if early_stop>=60 else epoch+1

        self.tb.close #Tensorboard close
        return self

In [7]:
# now we load the data and start the training
# baseline experiment: model01
# input: x,y,object, output:u,v,p. No FV attributes.

train_data = torch.load('train.pickle')
val_data = torch.load('val.pickle')

path = 'IVE_Models/model01_00/'
trained = IVEModelfit(input_size = 3,
                      input_edge_feat = 3,
                      edge_feature_dim=[4,8,16,32,64,64,32,16],
                      num_filters=[8,16,32,64,64,32,16,8],
                      output_size = 3,
                      node_attr_num=0, edge_attr_num=0, FV_hidden_size=0,
                      device='cuda:0')
trained.fit(path, train_data, val_data, max_epoch=1000, bs=64, lr=0.002, weight_decay=0.002, verbose=True)

epoch= 0 	 time= 6.149606704711914 	 loss= 0.13185713574290275
epoch= 5 	 time= 39.047693490982056 	 loss= 0.06717705965042114
epoch= 10 	 time= 71.73647475242615 	 loss= 0.048442137092351914
epoch= 15 	 time= 104.58683347702026 	 loss= 0.04115691296756267
epoch= 20 	 time= 137.26012134552002 	 loss= 0.035663222074508664
epoch= 25 	 time= 169.8927869796753 	 loss= 0.03106556922197342
epoch= 30 	 time= 202.71270513534546 	 loss= 0.026114320792257787
epoch= 35 	 time= 235.30182552337646 	 loss= 0.02300059374421835
epoch= 40 	 time= 267.9455683231354 	 loss= 0.019582093693315983
epoch= 45 	 time= 300.62061762809753 	 loss= 0.017927758749574422
epoch= 50 	 time= 333.2841913700104 	 loss= 0.017654642388224603
epoch= 55 	 time= 365.82870626449585 	 loss= 0.017357175033539535
epoch= 60 	 time= 398.45350670814514 	 loss= 0.01671259293332696
epoch= 65 	 time= 431.0202729701996 	 loss= 0.01666948936879635
epoch= 70 	 time= 463.6414883136749 	 loss= 0.016080802138894797
epoch= 75 	 time= 496.2992

In [ ]:
# now we load the data and start the training
# SAF and dSDF experiment: model02
# input: x,y,SAF,dSDF, output:u,v,p. No FV attributes.

train_data = torch.load('train2.pickle')
val_data = torch.load('val2.pickle')

path = 'IVE_Models/model02_TEST00/'
trained = IVEModelfit(input_size = 12,
                      input_edge_feat = 12,
                      edge_feature_dim=[4,8,16,32,64,64,32,16],
                      num_filters=[8,16,32,64,64,32,16,8],
                      output_size = 3,
                      node_attr_num=0, edge_attr_num=0, FV_hidden_size=0,
                      device='cuda:0')
trained.fit(path, train_data, val_data, max_epoch=1000, bs=64, lr=0.002, weight_decay=0.002, verbose=True)

/ntuzfs/jessica/anaconda3/envs/cu113py39/lib/python3.9/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


epoch= 0 	 time= 6.798266410827637 	 loss= 0.12525332629680633
epoch= 5 	 time= 41.412795066833496 	 loss= 0.0595178634673357
epoch= 10 	 time= 75.85290288925171 	 loss= 0.04240341477096081
epoch= 15 	 time= 110.55145764350891 	 loss= 0.027491466663777827
epoch= 20 	 time= 145.94313669204712 	 loss= 0.022698229737579824
epoch= 25 	 time= 180.5417604446411 	 loss= 0.021078452430665494
epoch= 30 	 time= 214.50816679000854 	 loss= 0.018639970235526562
epoch= 35 	 time= 248.8865180015564 	 loss= 0.016500208023935557
epoch= 40 	 time= 283.21240186691284 	 loss= 0.014299478940665722
epoch= 45 	 time= 318.8955397605896 	 loss= 0.013248862009495497
epoch= 50 	 time= 353.99534249305725 	 loss= 0.013119911532849074
epoch= 55 	 time= 388.9277377128601 	 loss= 0.012882695086300373
epoch= 60 	 time= 422.9955401420593 	 loss= 0.012236639000475407
epoch= 65 	 time= 458.3000657558441 	 loss= 0.012375181633979082
epoch= 70 	 time= 492.57055592536926 	 loss= 0.01232894668355584
epoch= 75 	 time= 526.187

In [3]:
#test and print the test MAE loss

def testIVE(model, dataset, device='cuda:1'):
    model = model.to(device); model.eval()
    criterion = nn.L1Loss().to(device)
    testLoader = DataLoader(dataset, batch_size=1, shuffle=False)
    torch.set_grad_enabled(False)

    running_loss = 0; num_batches = 0
    for batch_list in testLoader:
        batch_list = batch_list.to(device)
        outputs, GTy = model(batch_list)
        loss = criterion(outputs, GTy)
        # add the loss of this batch to the running loss
        running_loss += loss.detach().item(); num_batches+=1

    # compute stats for the full training set
    total_loss = running_loss/num_batches
    torch.set_grad_enabled(True)
    return total_loss

In [9]:
test_data = torch.load('test.pickle')
best_model = torch.load('IVE_Models/model01/IVEModel_checkpoint_409.txt')

test_loss = testIVE(best_model, test_data, device='cuda:1')
print('test_loss: ',test_loss)

model_parameters = filter(lambda p: p.requires_grad, best_model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print('number of parmeters: ',params)

/home/jessica/anaconda3/envs/airfrans_env/lib/python3.9/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


test_loss:  0.011823220876976848
number of parmeters:  217853


In [8]:
test_data = torch.load('test2.pickle')
best_model = torch.load('IVE_Models/model02/IVEModel_checkpoint_506.txt')

test_loss = testIVE(best_model, test_data, device='cuda:1')
print('test_loss: ',test_loss)

model_parameters = filter(lambda p: p.requires_grad, best_model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print('number of parmeters: ',params)

/ntuzfs/jessica/anaconda3/envs/cu113py39/lib/python3.9/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


test_loss:  0.00660703303758055
number of parmeters:  222461


**Model3**
---

In [ ]:
#SIMPLEST ONE: just concatenate the edge attributes

In [12]:
# Build the network as described in (https://github.com/cfl-minds/gnn_laminar_flow)
# but using pyTorch

class swish(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
    def forward(self, x):
        return x * torch.sigmoid(x)

###################################
## InvariantEdgeFVConv CLASS DEF ##
###################################
#@title InvariantEdgeFVConv codes, with node_attr_num and edge_attr_num for FV
class InvariantEdgeFVConv(nng.conv.MessagePassing): #InvariantEdgeConv(nng.conv.GCNConv):
    '''Convolutions class as described in paper, built on GCNConv, 
    with SGCN and FVnew adaptations'''
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        #aggr: Optional[Union[str, List[str], Aggregation]] = 'add',

        old_mlp_ief: int = 0, #paper: previous edge feature dimension
        mlp_hs: int = 128, #paper: two MLPs with hidden size 128 used
        mlp_ief: int = 1, #paper: one MLP with intermediate edge feature dimension fe
        
        node_attr_num: int = 1, #FVnew: number of persistent node_attr
        edge_attr_num: int = 6, #FVnew: number of edge_attr + rel_node_attr
        # hidden_size: int = 0, #sGCN: convolution internal hidden size
        dropout: float = 0, #sGCN: dropout
        **kwargs
    ):

        kwargs.setdefault('aggr', 'add')
        super().__init__(**kwargs)

        self.in_channels = in_channels + node_attr_num
        self.out_channels = out_channels
        
        #paper####
        # self.mlp_message = nng.MLP([in_channels*2+old_mlp_ief, mlp_hs, mlp_ief],act='swish', bias=True)
        # self.mlp_update = nng.MLP([mlp_ief+in_channels, mlp_hs, out_channels],act='swish', bias=True)
        self.mlp_message = torchvision.ops.MLP(self.in_channels*2+old_mlp_ief+edge_attr_num, [mlp_hs, mlp_ief], \
                                               activation_layer=swish, bias=True)
        self.mlp_update = torchvision.ops.MLP(mlp_ief+self.in_channels, [mlp_hs, out_channels], \
                                              activation_layer=swish, bias=True)
        self.new_edge_features = None #torch.tensor([0.])

        #FVnew or sGCN####
        # self.lin_in = torch.nn.Linear(edge_attr_num, hidden_size * in_channels)
        # self.lin_in = torch.nn.Linear(edge_attr_num, mlp_ief)
        # self.lin_out = torch.nn.Linear(hidden_size * in_channels, out_channels)
        # self.dropout = dropout ####

    #def forward(self, x: Tensor, edge_index: Adj, edge_weight: OptTensor = None) -> Tensor:
    def forward(self, x, edge_index, edge_feat=None, edge_attr=None, node_attr=None):

        if (node_attr is not None): #for FVnew
            x = torch.cat((x,node_attr),dim=1) #persistent node_attr.

        # if edge_feat is None: #for paper: previous edge feat added if existent
        #     edge_feat = torch.zeros((edge_index.size(1),1))
        out = self.propagate(x=x, edge_index=edge_index, edge_feat=edge_feat, edge_attr=edge_attr) #for FVnew

        return out, self.new_edge_features

    #def message(self, x_j: Tensor, edge_weight: OptTensor) -> Tensor:
    #   return x_j if edge_weight is None else edge_weight.view(-1, 1) * x_j
    def message(self, x_j, x_i, edge_feat, edge_attr=None):
        """
        x_j [num_edges, label_dim]
        x_i [num_edges, label_dim]
        edge_attr [num_edges, #attr] or none ##ADDED
        """
        symmetric = (x_j + x_i)/2 #paper: symmetric node features
        asymmetric = abs(x_j - x_i)/2 #instead of raw features x_j and x_i
        if edge_feat is not None:
            if edge_attr is not None:
                new_edge_feat = torch.cat((symmetric,asymmetric,edge_feat,edge_attr),dim=1)
            else:
                new_edge_feat = torch.cat((symmetric,asymmetric,edge_feat),dim=1)
        else:
            new_edge_feat = torch.cat((symmetric,asymmetric),dim=1)
        message = self.mlp_message(new_edge_feat)
        
        # message = message * F.sigmoid(message) #MLP swish activation if act was None
        self.new_edge_features = message

        ##for FVnew ####
        # if edge_attr is not None: 
        #   scaling = F.relu(self.lin_in(edge_attr))  # [n_edges, hidden_size * in_channels]
        # else:
        #   scaling = torch.ones(x_j.size()).unsqueeze(-1).to(x_j.device)
        # n_edges = x_j.size(0)
        # # [n_edges, in_channels, ...] * [n_edges, in_channels, 1]
        # # result = scaling.reshape(n_edges, self.in_channels, -1) * message.unsqueeze(-1)
        # result = scaling.reshape(n_edges, -1) * message
        ####

        return message #result.view(n_edges, -1)

    #def message_and_aggregate(self, adj_t: SparseTensor, x: Tensor) -> Tensor:
    #    return spmm(adj_t, x, reduce=self.aggr)

    def update(self, aggr_out, x):
        """
        x_j [num_nodes, label_dim, 1]
        aggr_out [num_nodes, label_dim, out_channels]
        """
        aggr_out = torch.cat((x,aggr_out),dim=1) #paper
        aggr_out = self.mlp_update(aggr_out)
        # aggr_out = aggr_out * F.sigmoid(aggr_out) #MLP swish activation if act was None

        ##for sGCN ####
        # aggr_out = self.lin_out(aggr_out)  #[num_nodes, label_dim, out_features]
        # aggr_out = torch.tanh(aggr_out); #aggr_out = F.relu(aggr_out)
        # aggr_out = F.dropout(aggr_out, p=self.dropout, training=self.training)
        ####

        return aggr_out

In [9]:
# now we load the data and start the training
# FV experiment: model02
# input: x,y,SAF,dSDF, output:u,v,p. WITH FV attributes.

train_data = torch.load('train3.pickle')
val_data = torch.load('val3.pickle')

path = 'IVE_Models/model03c_00/'
trained = IVEFVModelfit(input_size = 12,
                      input_edge_feat = 12,
                      edge_feature_dim=[4,8,16,32,64,64,32,16],
                      num_filters=[8,16,32,64,64,32,16,8],
                      output_size = 3,
                      node_attr_num=1,
                      edge_attr_num=6,
                      device='cuda:1')
trained.fit(path, train_data, val_data, max_epoch=1000, bs=64, lr=0.002, weight_decay=0.002, verbose=True)

/home/jessica/anaconda3/envs/lim_wp5/lib/python3.9/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


epoch= 0 	 time= 15.893697738647461 	 loss= 0.1820575028657913
epoch= 5 	 time= 69.43811535835266 	 loss= 0.07708506971597671
epoch= 10 	 time= 118.29100823402405 	 loss= 0.0685136365890503
epoch= 15 	 time= 167.4692735671997 	 loss= 0.061980309933423995
epoch= 20 	 time= 216.03236889839172 	 loss= 0.05479741886258125
epoch= 25 	 time= 261.13044691085815 	 loss= 0.0505585926771164
epoch= 30 	 time= 306.0433759689331 	 loss= 0.047943304926157
epoch= 35 	 time= 351.378751039505 	 loss= 0.03491306498646736
epoch= 40 	 time= 395.67210483551025 	 loss= 0.030647451430559157
epoch= 45 	 time= 441.6814298629761 	 loss= 0.02928360678255558
epoch= 50 	 time= 487.191609621048 	 loss= 0.02701968662440777
epoch= 55 	 time= 531.8305928707123 	 loss= 0.02537402592599392
epoch= 60 	 time= 578.0391874313354 	 loss= 0.022810547053813933
epoch= 65 	 time= 624.734338760376 	 loss= 0.022880838662385942
epoch= 70 	 time= 669.6569032669067 	 loss= 0.020208725035190583
epoch= 75 	 time= 715.3023483753204 	 lo

In [6]:
test_data = torch.load('test3.pickle')
best_model = torch.load('IVE_Models/model03c_00/IVEModel_checkpoint_460.txt')

test_loss = testIVE(best_model, test_data, device='cuda:1')
print('test_loss: ',test_loss)

model_parameters = filter(lambda p: p.requires_grad, best_model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print('number of parmeters: ',params)

/home/jessica/anaconda3/envs/lim_wp5/lib/python3.9/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


test_loss:  0.008490977995097636
number of parmeters:  231677
